# Notebook de cas d'utilisation de Dask avec un TIF

Dans ce notebook, nous cherchons à utiliser Dask et ses modules sur des TIF afin de faire du calcul distribué.

Nous proposons ici un exemple simple : calcul de NDVI sur ces TIF.

Les API utilisées sont également simples, pas d'affichage cartographique ici, on utilise seulement les dask.array, un équivelent des numpy.array en version distribuée. Nous chargerons donc les TIF sous forme de tableaux multidimensionnels et réaliserons le calcul de NDVI.


## Initialisation du client et du cluster

Nous pouvons utiliser le cluster TREX en créant 2 workers avec 2 cpus et 16GB de RAM

In [ ]:
# Your code

In [ ]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress

account_TREX ='formation_isae'
partition_Trex = "cpu19_rh8"
qos_Trex = "--qos=cpu_2019_40"


cluster = SLURMCluster(
    # Dask-worker specific keywords
    n_workers=2,                # start 2 workers
    cores=2,                    # each worker runs on 2 cores
    memory="8GB",              # each worker uses 8GB memory (on TREX g2022 : nb_cores*8Go, on g2019 : nb_cores*4.6Go )
    processes=1,                # Number of Python processes to cut up each job
    local_directory='$TMPDIR',  # Location to put temporary data if necessary
    account=account_TREX,
    queue = partition_Trex,
    walltime='01:00:00',
    interface='ib0',
    log_directory='../dask-logs',
    job_extra_directives=[qos_Trex] # qos to use
)

cluster

In [ ]:
client = Client(cluster)
client

## Initialisation des modules

On récupère les modules nécessaires

In [17]:
from dask.array.image import imread
import dask.array as da
import matplotlib.pyplot as plt

## Définition des fonctions utilisées

La fonction realizeNDVI va réaliser le calcul de NDVI (différence normalisée) sur le TIF, c'est une simple opération de type tableau numpy, réalisable avec des tableaux dask.

In [18]:
def realizeNDVI(inputImage):
    return (inputImage[:, :, :, 2] - inputImage[:, :, :, 3]) / (inputImage[:, :, :, 3] + inputImage[:, :, :, 2])

## Récupération des TIF

Nous récupérons les TIFs suivant pour ce notebook:

In [19]:
!ls -lh /work/datalake/pangeo_demo/rt_algos/N1/

total 4.5G
-rwxrwxr-x+ 1 geninv cneshpc 565M May 13  2020 SPOT5_HRG2_XS_20150411_N1_TUILE_ArlesFranceD0000B0000.TIF
-rwxrwxr-x+ 1 geninv cneshpc 565M May 13  2020 SPOT5_HRG2_XS_20150511_N1_TUILE_ArlesFranceD0000B0000.TIF
-rwxrwxr-x+ 1 geninv cneshpc 565M May 13  2020 SPOT5_HRG2_XS_20150605_N1_TUILE_ArlesFranceD0000B0000.TIF
-rwxrwxr-x+ 1 geninv cneshpc 565M May 13  2020 SPOT5_HRG2_XS_20150720_N1_TUILE_ArlesFranceD0000B0000.TIF
-rwxrwxr-x+ 1 geninv cneshpc 565M May 13  2020 SPOT5_HRG2_XS_20150725_N1_TUILE_ArlesFranceD0000B0000.TIF
-rwxrwxr-x+ 1 geninv cneshpc 565M May 13  2020 SPOT5_HRG2_XS_20150809_N1_TUILE_ArlesFranceD0000B0000.TIF
-rwxrwxr-x+ 1 geninv cneshpc 565M May 13  2020 SPOT5_HRG2_XS_20150819_N1_TUILE_ArlesFranceD0000B0000.TIF
-rwxrwxr-x+ 1 geninv cneshpc 563M May 13  2020 SPOT5_HRG2_XS_20150903_N1_TUILE_ArlesFranceD0000B0000.TIF


## Préparation du Dask.array contenant les TIF

L'API dask array propose une méthode pour lire un ensemble de fichier TIF.
Afin de simuler un plus grand nombre de fichier, nous allons concaténer plusieurs fois le même array en mémoire. Utilisation de la fonction `imread`

In [ ]:
# Your code 
# images_array 

In [ ]:
images_array = imread('/work/datalake/pangeo_demo/rt_algos/N1/*')
images_array

Imaginons donc que nous avons plus de 8 images en duplicant les tableaux (multiplication "articielle" du nb images) :

In [ ]:
big_images_array = da.concatenate([images_array] * 10)
big_images_array

Il est possible de retuiler les images si le calcul est complexe et à besoin de chunks plus petit.
Dans le cas présent on va limiter, car ce n'est pas vraiment la peine.

In [ ]:
big_images_array = big_images_array.rechunk((1, 4050, 4550, 4))
big_images_array

## Calcul du NDVI et affichage

On réalise le calcul de NDVI sur les différentes images de manière distribuée

In [24]:
ndviTIF = realizeNDVI(big_images_array)

Calculer et afficher le résultat sur trois images :

In [ ]:
# Your code

In [32]:
april = ndviTIF[0].compute()
june = ndviTIF[2].compute()
august = ndviTIF[6].compute()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(13, 3), ncols=3)
ax1.set_title('2015/04/11')
ax2.set_title('2015/06/05')
ax3.set_title('2015/08/19')

pos = ax1.imshow(april)
pos1 = ax2.imshow(june)
pos2 = ax3.imshow(august)
fig.colorbar(pos, ax=ax3, label="NVDI")

On peut aussi calculer la moyenne du NDVI sur chaque image, attention à ignorer les nan à cause de potentielles division par zero.

Forcément le résultat sera cyclique vu qu'on répète toujours les 8 mêmes images.

In [ ]:
times = 2
nb_tiff = times * 8

meanNdviTIF = da.nanmean(ndviTIF[0:nb_tiff], axis=(1,2))

plt.plot(meanNdviTIF.compute())

In [34]:
client.close()
cluster.close()